<a href="https://colab.research.google.com/github/sureshmecad/Google-Colab/blob/master/1_pipeline_scaler_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
seol = pd.read_csv("/content/drive/MyDrive/TMLC/Seoul-Bike-Trip-Duration.zip")
seol.head()

,Unnamed: 0,Duration,Distance,PLong,PLatd,DLong,DLatd,Haversine,Pmonth,Pday,Phour,Pmin,PDweek,Dmonth,Dday,Dhour,Dmin,DDweek,Temp,Precip,Wind,Humid,Solar,Snow,GroundTemp,Dust
0,0,3,50,37.544666,126.888359,37.544666,126.888359,0.000000,1,1,0,0,0,1,1,0,4,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
1,1,24,7670,37.506199,127.003944,37.551250,127.035103,5.713529,1,1,0,0,0,1,1,0,25,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
2,2,8,1390,37.544590,127.057083,37.537014,127.061096,0.913702,1,1,0,0,0,1,1,0,9,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
3,3,8,1820,37.571102,127.023560,37.561447,127.034920,1.468027,1,1,0,1,0,1,1,0,10,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0
4,4,4,850,37.573242,127.015907,37.565849,127.016403,0.823227,1,1,0,1,0,1,1,0,6,0,-3.2,0.0,0.5,40.0,0.0,0.0,-2.2,25.0


In [ ]:
Bike_original = seol.copy()

In [ ]:
df = seol.drop(['PLatd', 'DLatd', 'Pday', 'Pmin', 'Dday', 'Dmin', 'Precip', 'Snow', 'Dust'], axis=1)
df.head()

,Unnamed: 0,Duration,Distance,PLong,DLong,Haversine,Pmonth,Phour,PDweek,Dmonth,Dhour,DDweek,Temp,Wind,Humid,Solar,GroundTemp
0,0,3,50,37.544666,37.544666,0.000000,1,0,0,1,0,0,-3.2,0.5,40.0,0.0,-2.2
1,1,24,7670,37.506199,37.551250,5.713529,1,0,0,1,0,0,-3.2,0.5,40.0,0.0,-2.2
2,2,8,1390,37.544590,37.537014,0.913702,1,0,0,1,0,0,-3.2,0.5,40.0,0.0,-2.2
3,3,8,1820,37.571102,37.561447,1.468027,1,0,0,1,0,0,-3.2,0.5,40.0,0.0,-2.2
4,4,4,850,37.573242,37.565849,0.823227,1,0,0,1,0,0,-3.2,0.5,40.0,0.0,-2.2


In [ ]:
# Remove all duplicates
df.drop_duplicates(inplace=True)
print("Dataset size before removing duplicates:", Bike_original.shape)
print("Dataset size after removing duplicates:", df.shape)

Dataset size before removing duplicates: (9601139, 26)
Dataset size after removing duplicates: (9601139, 17)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [ ]:
df = df.iloc[0:900000, 0:16]

In [ ]:
df.head()

,Unnamed: 0,Duration,Distance,PLong,DLong,Haversine,Pmonth,Phour,PDweek,Dmonth,Dhour,DDweek,Temp,Wind,Humid,Solar
0,0,3,50,37.544666,37.544666,0.000000,1,0,0,1,0,0,-3.2,0.5,40.0,0.0
1,1,24,7670,37.506199,37.551250,5.713529,1,0,0,1,0,0,-3.2,0.5,40.0,0.0
2,2,8,1390,37.544590,37.537014,0.913702,1,0,0,1,0,0,-3.2,0.5,40.0,0.0
3,3,8,1820,37.571102,37.561447,1.468027,1,0,0,1,0,0,-3.2,0.5,40.0,0.0
4,4,4,850,37.573242,37.565849,0.823227,1,0,0,1,0,0,-3.2,0.5,40.0,0.0


In [ ]:
X = df.drop('Duration', axis=1)
y = df['Duration']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

### **Ridge Regressor**

In [ ]:
ridge = Pipeline([
    ("scaler", StandardScaler()),
    ("regressor", Ridge(alpha=1e-6))])

ridge.fit(X_train, y_train)

y_pred_ridge = ridge.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred_ridge))

print("MAE:", mean_absolute_error(y_test, y_pred_ridge))

MSE: 254.19051186477066
MAE: 9.5440514971741


In [ ]:
print("Ridge : Train Score {:.2f} & Test Score {:.2f}".format(ridge.score(X_train, y_train), ridge.score(X_test, y_test)))

Ridge : Train Score 0.55 & Test Score 0.56


### **XGBoost Regressor**

In [ ]:
XGB = Pipeline([("scaler", StandardScaler()), ("regressor", XGBRegressor())])

XGB.fit(X_train, y_train)

y_pred_xgb = XGB.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred_xgb))

print("MAE:", mean_absolute_error(y_test, y_pred_xgb))

[12:43:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MSE: 145.38062964784226
MAE: 6.9525592897163495


In [ ]:
print("XGBoost : Train Score {:.2f} & Test Score {:.2f}".format(XGB.score(X_train, y_train), XGB.score(X_test, y_test)))

XGBoost : Train Score 0.75 & Test Score 0.75


### **LGBMRegressor**

In [ ]:
LGB = Pipeline([("scaler", StandardScaler()), ("regressor", LGBMRegressor())])

LGB.fit(X_train, y_train)

y_pred_lgb = LGB.predict(X_test)

print("MSE:", mean_squared_error(y_test, y_pred_lgb))

print("MAE:", mean_absolute_error(y_test, y_pred_lgb))

MSE: 105.02246503691852
MAE: 6.109513358511218


In [ ]:
print("LGBM : Train Score {:.2f} & Test Score {:.2f}".format(LGB.score(X_train, y_train), LGB.score(X_test, y_test)))

LGBM : Train Score 0.82 & Test Score 0.82


--------------------------------------